# **Logistic Regression**

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import collections
import pickle
from tqdm import tqdm
import pylab
use_colab = True
if use_colab:
  from google.colab import drive
import tensorflow as tf
TINY = 1e-30
EPS = 1e-4
nax = np.newaxis
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
if use_colab:
  drive_name = '/content/drive'
  drive.mount(drive_name)
  drive_413_A1_folder = 'Queens_data_task'
  drive_location = drive_name + '/My Drive/' + drive_413_A1_folder  # Change this to where your files are located
else:
  # set the drive_location variable to whereever the extracted contents are.
  drive_location = ''

# Hyper-parameters 
input_size = 4096   # 784
num_classes = 10
num_epochs = 100
batch_size = 100
learning_rate = 0.001

# MNIST dataset (images and labels)
Y = np.loadtxt(drive_location + '/' +"train_y.csv", delimiter=",")
Y = Y.reshape(-1)
Y = np.int_(Y)
X = np.load(drive_location + '/' + 'X.npy')
X = torch.Tensor(X)
Y = torch.tensor(Y, dtype=torch.int64)
X = X.reshape(-1, input_size)
# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
#total_step = len(train_loader)
for epoch in range(num_epochs):
    
    for i in range(50000//batch_size):
        # Reshape images to (batch_size, input_size)
        tempX = X[i*batch_size: i*batch_size + batch_size]
        tempY = Y[i*batch_size: i*batch_size + batch_size]
        
        # Forward pass
        outputs = model(tempX)
        loss = criterion(outputs, tempY)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
    

with torch.no_grad():
    correct = 0
    total = 0
    
    images = X.reshape(-1, input_size)
    outputs = model(X)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == Y).sum()

    print('Accuracy of the model on the 50000 test images: {} %'.format(100 * correct / 50000))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Accuracy of the model on the 10000 test images: 17 %


As we can see, the logistic regression model fails to classify the images and has a test accuracy of 17 % only. This is because detecting the biggest digit in the image is a highly non-linear problem. Logistic regression models have (piecewise) linear decision boundaries so they are not complex enough to learn the features in the training images.

#  **Convolutional neural networks**

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import collections
import pickle
from tqdm import tqdm
import pylab
use_colab = True
if use_colab:
  from google.colab import drive
import tensorflow as tf
TINY = 1e-30
EPS = 1e-4
nax = np.newaxis
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
if use_colab:
  drive_name = '/content/drive'
  drive.mount(drive_name)
  drive_413_A1_folder = 'Queens_data_task'
  drive_location = drive_name + '/My Drive/' + drive_413_A1_folder  # Change this to where your files are located
else:
  # set the drive_location variable to whereever the extracted contents are.
  drive_location = ''




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import random
# Hyper-parameters 
input_size = 4096   # 784
num_classes = 10
num_epochs = 35
batch_size = 100
learning_rate = 0.1
momentum = 0.8

class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
          
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
          
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(64 * 16 * 16, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 10),
        )
          
        for m in self.features.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
        for m in self.classifier.children():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform(m.weight)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        
        return x     


# MNIST dataset (images and labels)
Y = np.loadtxt(drive_location + '/' +"train_y.csv", delimiter=",")
Y = Y.reshape(-1)
Y = np.int_(Y)
X = np.load(drive_location + '/' + 'X.npy')
X = torch.Tensor(X).reshape(-1,1,64,64)
Y = torch.tensor(Y, dtype=torch.int64).type(torch.cuda.FloatTensor)
X = X.reshape(-1, input_size).type(torch.cuda.FloatTensor)

network = Net()
network.cuda()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
# Train the model
#total_step = len(train_loader)
network.train()
for epoch in range(num_epochs):
    for i in range(25000//batch_size):
        # Reshape images to (batch_size, input_size)
        tempX = X[i*batch_size: i*batch_size + batch_size].reshape(-1, 1,64,64)
        tempY = Y[i*batch_size: i*batch_size + batch_size].type(torch.int64)
        
        optimizer.zero_grad()
        output = network(tempX)
        loss = nn.CrossEntropyLoss()(output, tempY)
        loss.backward()
        optimizer.step()
    
        
    
        print ('Epoch [{}/{}], Loss: {:.4f}' 
              .format(epoch+1, num_epochs, loss.item()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Streaming output truncated to the last 5000 lines.
Epoch [16/35], Loss: 0.2538
Epoch [16/35], Loss: 0.5223
Epoch [16/35], Loss: 0.3861
Epoch [16/35], Loss: 0.4318
Epoch [16/35], Loss: 0.3757
Epoch [16/35], Loss: 0.4069
Epoch [16/35], Loss: 0.3106
Epoch [16/35], Loss: 0.3724
Epoch [16/35], Loss: 0.5502
Epoch [16/35], Loss: 0.2834
Epoch [16/35], Loss: 0.4431
Epoch [16/35], Loss: 0.5090
Epoch [16/35], Loss: 0.3455
Epoch [16/35], Loss: 0.5332
Epoch [16/35], Loss: 0.4055
Epoch [16/35], Loss: 0.4741
Epoch [16/35], Loss: 0.7823
Epoch [16/35], Loss: 0.3758
Epoch [16/35], Loss: 0.5677
Epoch [16/35], Loss: 0.4073
Epoch [16/35], Loss: 0.4730
Epoch [16/35], Loss: 0.2705
Epoch [16/35], Loss: 0.5075
Epoch [16/35], Loss: 0.2985
Epoch [16/35], Loss: 0.3265
Epoch [16/35], Loss: 0.3391
Epoch [16/35], Loss: 0.4629
Epoch [16/35], Loss: 0.4519
Epoch [16/35], Loss: 0.3134
Epoch [16/35], Loss: 0.5209
Epoch [16/35], Loss: 0.5930
Epoch [16/35], Loss: 0.3701
Epoch [16/35], Loss: 0.3153
Epoch [16/35], Loss: 0.79

RuntimeError: ignored

In [0]:
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(0,25): #since GPU does not have enough memory
      images = X[25000 + 1000*i:25000 + 1000*i + 1000].reshape(-1,1,64,64)
      outputs = network(images)
      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == Y[25000 + 1000*i:25000 + 1000*i + 1000]).sum()

    print('Accuracy of the model on the 25000 test images: {} %'.format(100 * correct / 25000))


Accuracy of the model on the 25000 test images: 77 %


Our CNN consists of 4 convolution layers, each separated with a non-linearity (ReLU) in addition to Max pooling and Batch normalization to speed up training. The output of the CNN is then passed on to classification is which basically a simple feed forward NN. As we can see, the CNN performs much better on this task. This because we have introduced non-linearity to our network across multiple layers (unlike LR), in addition to convolution and Max pooling which allow us to efficiently learn low- and high-level features in the image.
We were able to achieve a test accuracy of 77%, a higher accuracy can be achieved if we tune the hyper-parameters even further.


# **RNN**

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# Hyper-parameters 
# Hyper-parameters
sequence_length = 64
input_size = 64
hidden_size = 164
num_layers = 2
num_classes = 10
batch_size = 2500
num_epochs = 100
learning_rate = 0.001

# MNIST dataset (images and labels)
Y = np.loadtxt(drive_location + '/' +"train_y.csv", delimiter=",")
Y = Y.reshape(-1)
Y = np.int_(Y)
X = np.load(drive_location + '/' + 'X.npy')
X = torch.Tensor(X).reshape(-1,1,64,64)
Y = torch.tensor(Y, dtype=torch.int64).type(torch.cuda.FloatTensor)
X = X.reshape(-1, 4096).type(torch.cuda.FloatTensor)



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)



criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Train the model
#model.train()
for epoch in range(num_epochs):
    for i in range(25000//batch_size):
        # Reshape images to (batch_size, input_size)
        tempX = X[i*batch_size: i*batch_size + batch_size].reshape(-1,64,64)
        tempY = Y[i*batch_size: i*batch_size + batch_size].type(torch.int64)
        
        optimizer.zero_grad()
        output = model(tempX)
        loss = criterion(output, tempY)
        loss.backward()
        optimizer.step()
    
        
    
        print ('Epoch [{}/{}], Loss: {:.4f}' 
              .format(epoch+1, num_epochs, loss.item()))

Epoch [1/100], Loss: 10.2922
Epoch [1/100], Loss: 7.3451
Epoch [1/100], Loss: 8.9212
Epoch [1/100], Loss: 8.1663
Epoch [1/100], Loss: 6.2698
Epoch [1/100], Loss: 5.3882
Epoch [1/100], Loss: 4.3179
Epoch [1/100], Loss: 3.9329
Epoch [1/100], Loss: 3.6634
Epoch [1/100], Loss: 3.3646
Epoch [2/100], Loss: 3.2213
Epoch [2/100], Loss: 2.9136
Epoch [2/100], Loss: 2.9703
Epoch [2/100], Loss: 2.8979
Epoch [2/100], Loss: 2.8136
Epoch [2/100], Loss: 2.7862
Epoch [2/100], Loss: 2.7498
Epoch [2/100], Loss: 2.7214
Epoch [2/100], Loss: 2.6757
Epoch [2/100], Loss: 2.5875
Epoch [3/100], Loss: 2.5792
Epoch [3/100], Loss: 2.5050
Epoch [3/100], Loss: 2.5413
Epoch [3/100], Loss: 2.5386
Epoch [3/100], Loss: 2.5294
Epoch [3/100], Loss: 2.5115
Epoch [3/100], Loss: 2.4887
Epoch [3/100], Loss: 2.4814
Epoch [3/100], Loss: 2.4681
Epoch [3/100], Loss: 2.4567
Epoch [4/100], Loss: 2.4439
Epoch [4/100], Loss: 2.4138
Epoch [4/100], Loss: 2.4470
Epoch [4/100], Loss: 2.4268
Epoch [4/100], Loss: 2.4134
Epoch [4/100], Loss

In [0]:
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(0,25): #since GPU does not have enough memory
      images = X[25000 + 1000*i:25000 + 1000*i + 1000].reshape(-1,64,64)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == Y[25000 + 1000*i:25000 + 1000*i + 1000]).sum()

    print('Accuracy of the model on the 25000 test images: {} %'.format(100 * correct / 25000))

Accuracy of the model on the 25000 test images: 10 %


While RNNs do have non-linearities which enable them to learn complex features, they are mainly designed for tasks that have variable length inputs like speeach recognition or machine translation. This classification task requires recognizing patterns across space which CNNs can do very well in. RNNs, however, are designed to build up memory so they can learn from previous inputs. In other words, RNNs can perform well in problems that involve temporal locality which CNNs do well in tasks with spatial locality. 

It is clear that using an RNN for this task is no better that randomly guessing even after training for 100 epochs with batch size 2500!

# **Gated CNN**

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# Hyper-parameters 
input_size = 64
hidden_size = 164
num_channels = 15
num_classes = 10
batch_size = 100
num_epochs = 100
learning_rate = 0.01

# MNIST dataset (images and labels)
Y = np.loadtxt(drive_location + '/' +"train_y.csv", delimiter=",")
Y = Y.reshape(-1)
Y = np.int_(Y)
X = np.load(drive_location + '/' + 'X.npy')
X = torch.Tensor(X).reshape(-1,1,64,64)
Y = torch.tensor(Y, dtype=torch.int64).type(torch.cuda.FloatTensor)
X = X.reshape(-1, 4096).type(torch.cuda.FloatTensor)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class GLUblock(nn.Module):
    def __init__(self, k, in_c, out_c):
        super(GLUblock, self).__init__()
        #only need to change shape of the residual if num_channels changes (i.e. in_c != out_c)
        if in_c == out_c:
            self.use_proj=0
        else:
            self.use_proj=1
        self.convresid=nn.utils.weight_norm(nn.Conv2d(in_c, out_c, kernel_size=(1,1)),name='weight')
        
        
        self.convx1b = nn.utils.weight_norm(nn.Conv2d(in_c, out_c, kernel_size=k,padding=k//2),name='weight')
        self.convx2b = nn.utils.weight_norm(nn.Conv2d(in_c, out_c, kernel_size=k,padding=k//2),name='weight')
        

    def forward(self, x):
        residual = x
        if self.use_proj==1:# if in_c != out_c, need to change size of residual
            residual=self.convresid(residual)
        
        x1 = self.convx1b(x) 
        x2 = self.convx2b(x) 
        x2 = torch.sigmoid(x2)
        x=torch.mul(x1,x2) 
        return x+residual

class GCNN(nn.Module):
  def __init__(self, k, in_c, out_c):
    super(GCNN, self).__init__()
    self.GCN_layers = nn.Sequential(
        GLUblock(k, in_c, out_c),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(out_c),
        GLUblock(k, out_c, out_c),
        nn.MaxPool2d(kernel_size=2),
        nn.BatchNorm2d(out_c),
        GLUblock(k, out_c, out_c)
    )
    self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(16 * 16 * out_c, 16*16),
            nn.BatchNorm1d(16 * 16),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(16 * 16, 50),
            nn.BatchNorm1d(50),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(50, 10),
        )
  def forward(self, x):
    out = self.GCN_layers(x)
    out = out.view(out.size(0), -1)
    out = self.classifier(out)
    return out


model = GCNN(5, 1, num_channels).to(device)



criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Train the model
#model.train()
for epoch in range(num_epochs):
    for i in range(25000//batch_size):
        # Reshape images to (batch_size, input_size)
        tempX = X[i*batch_size: i*batch_size + batch_size].reshape(-1,1,64,64)
        tempY = Y[i*batch_size: i*batch_size + batch_size].type(torch.int64)
        
        optimizer.zero_grad()
        output = model(tempX)
        loss = criterion(output, tempY)
        loss.backward()
        optimizer.step()
    
        
    
        print ('Epoch [{}/{}], Loss: {:.4f}' 
              .format(epoch+1, num_epochs, loss.item()))


Streaming output truncated to the last 5000 lines.
Epoch [81/100], Loss: 0.1836
Epoch [81/100], Loss: 0.2435
Epoch [81/100], Loss: 0.1923
Epoch [81/100], Loss: 0.1696
Epoch [81/100], Loss: 0.2025
Epoch [81/100], Loss: 0.2578
Epoch [81/100], Loss: 0.3794
Epoch [81/100], Loss: 0.1507
Epoch [81/100], Loss: 0.4058
Epoch [81/100], Loss: 0.4040
Epoch [81/100], Loss: 0.0907
Epoch [81/100], Loss: 0.1893
Epoch [81/100], Loss: 0.2003
Epoch [81/100], Loss: 0.3039
Epoch [81/100], Loss: 0.1519
Epoch [81/100], Loss: 0.2716
Epoch [81/100], Loss: 0.3520
Epoch [81/100], Loss: 0.4349
Epoch [81/100], Loss: 0.2395
Epoch [81/100], Loss: 0.2358
Epoch [81/100], Loss: 0.2124
Epoch [81/100], Loss: 0.2015
Epoch [81/100], Loss: 0.4924
Epoch [81/100], Loss: 0.2137
Epoch [81/100], Loss: 0.2176
Epoch [81/100], Loss: 0.2161
Epoch [81/100], Loss: 0.2177
Epoch [81/100], Loss: 0.2190
Epoch [81/100], Loss: 0.2874
Epoch [81/100], Loss: 0.2930
Epoch [81/100], Loss: 0.1653
Epoch [81/100], Loss: 0.1905
Epoch [81/100], Loss:

In [0]:
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(25): #since GPU does not have enough memory
      images = X[25000 + 1000*i:25000 + 1000*i + 1000].reshape(-1,1,64,64)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == Y[25000 + 1000*i:25000 + 1000*i + 1000]).sum()

    print('Accuracy of the model on the 25000 test images: {} %'.format(100 * correct / 25000))

Accuracy of the model on the 25000 test images: 80 %


As we can see, gated CNNs even perform better on this task by achieving a test accuracy of 80 % (further training and tuning of hyper-parameters can yield higher accuracy). Gated linear units are a simple gating mechanism designed to tackle the vanishing gradient problem by having linear units coupled with gates. This retains the non-linear capabilities of the layers while allowing the gradient to propagate through the linear units. In other words, this technique allows some features to propagate through some layers, preventing the low-level features from vanishing deep into the network.